Load up experiment

In [1]:
import pandas as pd
import numpy as np
from data_reading.phyphox import get_experiments, read_experiment
from preprocessing._interpolation import align_data

experiments = get_experiments()
offsets = {}
with open("../../data/phyphox/short recordings/Juan/offset.txt") as f:
    for line in f:
       (key, val) = line.split(": ")
       offsets[key] = val

sample_rate = 50
data_frames = read_experiment(experiments[1], offsets=offsets)
data_frames = {key : align_data(data_frame, listening_rate=1000/sample_rate, reference_sensor=None) for key, data_frame in data_frames.items()}


/Users/julio/Documents/Uni/5_Master/Master Project/mp-mental-health-2020/src/preprocessing/_data_frame.py:31: FutureWarning: Passing datetime64-dtype data to TimedeltaIndex is deprecated, will raise a TypeError in a future version
  time_delta_index = pd.TimedeltaIndex(timestamp_to_date, unit=output_timestamp_unit)


Load up annotations

In [2]:
y = pd.read_csv("../../data/phyphox/short recordings/Juan/annotations.tsv", delimiter="\t", header=None)
y = y.iloc[:, [3,5,8]]
y.columns = ["start", "end", "label"]
# iterate over the annotations and split the timeseries in chunks
chunks = {}
for key, df in data_frames.items():
    chunks[key] = [df.iloc[int(annotation["start"]*sample_rate):int(annotation["end"]*sample_rate)] for i, annotation in y.iterrows()]

Preprocess data

In [3]:
# append the activity label (as int) and the action id to the dataframe
# we need to do this to be able to extract time series features later

labels = y.loc[:, "label"].unique()
label_ids = { l: i for l,i in zip(labels, range(0,len(labels))) }

y = y.replace(label_ids)
for i, cl in zip(range(len(y)), y.iterrows()):
    label_id = int(cl[1]["label"])
    action_id = cl[0]
    for hand, chunk_list in chunks.items():
        c = chunk_list[i]
        c["activity"] = label_id
        c["action_id"] = action_id

/Users/julio/opt/anaconda3/envs/master-project/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/Users/julio/opt/anaconda3/envs/master-project/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [4]:
chunks_concat_right = pd.concat(chunks["right"]).reset_index(drop=True)
chunks_concat_left = pd.concat(chunks["left"]).reset_index(drop=True)

Feature extraction

In [5]:
%load_ext autoreload
%autoreload 2

labels = y.loc[:, "label"].squeeze()
from features._timeseries_feature_extraction import extract_timeseries_features
features = extract_timeseries_features(chunks_concat_right)


Feature Extraction: 100%|██████████| 20/20 [00:34<00:00,  1.73s/it]


In [6]:
features.replace([np.inf, -np.inf], np.nan, inplace=True)
features.dropna(inplace=True, axis=1)

In [7]:
features

variable,acceleration x__abs_energy,acceleration x__absolute_sum_of_changes,"acceleration x__agg_autocorrelation__f_agg_""mean""__maxlag_40","acceleration x__agg_autocorrelation__f_agg_""median""__maxlag_40","acceleration x__agg_autocorrelation__f_agg_""var""__maxlag_40","acceleration x__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""max""","acceleration x__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""mean""","acceleration x__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""min""","acceleration x__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""var""","acceleration x__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""max""",...,linear_acceleration z__symmetry_looking__r_0.9500000000000001,linear_acceleration z__time_reversal_asymmetry_statistic__lag_1,linear_acceleration z__time_reversal_asymmetry_statistic__lag_2,linear_acceleration z__time_reversal_asymmetry_statistic__lag_3,linear_acceleration z__value_count__value_-1,linear_acceleration z__value_count__value_0,linear_acceleration z__value_count__value_1,linear_acceleration z__variance,linear_acceleration z__variance_larger_than_standard_deviation,linear_acceleration z__variation_coefficient
id,,,,,,,,,,,,,,,,,,,,,
0,3012.474218,89.593002,0.458670,0.376102,0.050137,2.380684,0.932232,-0.262859,1.206579,7.599889,...,1.0,-0.937369,1.611068,-0.568252,0.0,0.0,0.0,1.433567,1.0,-29.145071
1,6956.588974,131.413302,0.462771,0.433799,0.032429,8.930229,7.073781,5.202037,2.060620,10.434962,...,1.0,0.429494,-0.164980,0.597156,0.0,0.0,0.0,0.820118,0.0,-23.569820
2,8104.145071,133.492186,0.201058,0.188168,0.040767,8.197083,6.664591,5.545273,1.760806,10.093124,...,1.0,0.001389,0.280241,-0.246189,0.0,0.0,0.0,0.764085,0.0,-41.794496
3,4605.980768,90.490004,0.166964,0.151822,0.061212,5.171849,4.624486,4.201828,-0.403371,7.352635,...,1.0,0.762612,0.606624,-0.268059,0.0,0.0,0.0,2.633105,1.0,-12.511400
4,6726.574496,124.109812,0.530803,0.486180,0.027969,8.726986,6.704075,5.091302,1.793982,11.576229,...,1.0,0.031047,0.050746,0.043174,0.0,0.0,0.0,0.586616,0.0,-3.447960
5,37530.307701,112.825301,0.085717,0.205488,0.370038,10.456120,9.765932,9.264976,0.208281,11.114417,...,1.0,0.024199,-0.003485,-0.113954,0.0,0.0,0.0,0.751023,0.0,-2.155912
6,12310.916475,33.580562,0.082059,0.161694,0.398732,11.100144,9.971857,8.810321,0.538450,12.158878,...,1.0,-0.062139,-0.071586,-0.106532,0.0,0.0,0.0,2.832656,1.0,-5.377884
7,12993.208635,58.873554,0.628405,0.586513,0.020766,10.571669,8.934102,7.543843,1.221601,12.302055,...,1.0,-0.003978,-0.026251,-0.073435,0.0,0.0,0.0,2.056634,1.0,-3.122216
8,4958.295813,88.954265,0.213288,0.042454,0.123990,3.017792,2.193023,1.327690,0.439155,4.934755,...,1.0,-0.049494,-0.135443,-0.251444,0.0,0.0,0.0,2.483115,1.0,-3.116841


TODO: Feature selection & visualization

Train models and score results

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

models = [('Logistic Regression', LogisticRegression(solver='liblinear', multi_class='ovr')), ('LDA', LinearDiscriminantAnalysis()), ('LinearSVC', LinearSVC()), ('CART', DecisionTreeClassifier()), ('NB', GaussianNB())]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)
for name, model in models:
    print(name)
    scores = cross_val_score(model, X_scaled, labels, cv=3)
    print(scores.mean())





Logistic Regression
0.6413255360623782
LDA
0.912280701754386
LinearSVC
0.6052631578947368
CART
0.732943469785575
NB
0.6598440545808967


/Users/julio/opt/anaconda3/envs/master-project/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)
/Users/julio/opt/anaconda3/envs/master-project/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)
/Users/julio/opt/anaconda3/envs/master-project/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)
/Users/julio/opt/anaconda3/envs/master-project/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n